# Introduction

`pyiron_ontology` uses the `owlready2` library to build up pyiron-specific ontologies, and provides some extra tools to help you leverage these.

At present, the only ontology implemented is for the realm of atomistic calculations, and the scope of this ontology is still fairly limited.

First, let's import the ontology (an `owlready2.namespace.Ontology` object we define in the `pyiron_atomistics` repository on GitHub

In [1]:
from pyiron_ontology import atomistics_onto as onto

We can look at various properties of the ontology, just like other owl ontologies, e.g. the classes and individuals defined in this space:

In [2]:
list(onto.classes())

[pyiron_atomistics.PyObject,
 pyiron_atomistics.Parameter,
 pyiron_atomistics.InputParameter,
 pyiron_atomistics.OutputParameter,
 pyiron_atomistics.GenericParameter,
 pyiron_atomistics.Code,
 pyiron_atomistics.Label]

In [3]:
list(onto.individuals())

[pyiron_atomistics.Atomistic,
 pyiron_atomistics.lCode,
 pyiron_atomistics.DFT,
 pyiron_atomistics.MaterialProperty,
 pyiron_atomistics.PeriodicBoundaryConditions,
 pyiron_atomistics.UserInput,
 pyiron_atomistics.Bulk3dStructure,
 pyiron_atomistics.AtomisticEnergyCalculator,
 pyiron_atomistics.ChemicalElement,
 pyiron_atomistics.AtomicStructure,
 pyiron_atomistics.Executable,
 pyiron_atomistics.Flag,
 pyiron_atomistics.CreateStructureBulk,
 pyiron_atomistics.CreateStructureBulk/input/element,
 pyiron_atomistics.CreateStructureBulk/output/structure,
 pyiron_atomistics.CreateSurface,
 pyiron_atomistics.CreateSurface/input/element,
 pyiron_atomistics.CreateSurface/output/structure,
 pyiron_atomistics.Bulk_modulus,
 pyiron_atomistics.B_prime,
 pyiron_atomistics.Murnaghan,
 pyiron_atomistics.Murnaghan/output/equilibrium_bulk_modulus,
 pyiron_atomistics.Murnaghan/output/equilibrium_b_prime,
 pyiron_atomistics.Murnaghan/ref_job,
 pyiron_atomistics.EnergyCutoff,
 pyiron_atomistics.VASP,
 pyiro

We can make the usual owlready queries of these objects, e.g.

In [4]:
onto.VASP.domain

[pyiron_atomistics.Atomistic, pyiron_atomistics.lCode, pyiron_atomistics.DFT]

In [5]:
onto.VASP.is_a

[pyiron_atomistics.Code,
 pyiron_atomistics.has_conditions.only(owl.Nothing),
 pyiron_atomistics.has_transitive_conditions.only(owl.Nothing),
 pyiron_atomistics.has_options.only(OneOf([pyiron_atomistics.Bulk3dStructure, pyiron_atomistics.AtomisticEnergyCalculator])),
 pyiron_atomistics.is_in_domains.only(OneOf([pyiron_atomistics.Atomistic, pyiron_atomistics.lCode, pyiron_atomistics.DFT])),
 pyiron_atomistics.has_generic_parameter.only(OneOf([pyiron_atomistics.Executable])),
 pyiron_atomistics.has_input.only(OneOf([pyiron_atomistics.ENCUT, pyiron_atomistics.IBRION])),
 pyiron_atomistics.has_mandatory_input.only(OneOf([pyiron_atomistics.VASP/input/structure])),
 pyiron_atomistics.has_output.only(owl.Nothing)]

We can also look into some of the atomistics-specific relationships that have been defined:

In [6]:
onto.VASP.mandatory_input

[pyiron_atomistics.VASP/input/structure]

and we can chain these queries together in meaningful ways:

In [7]:
some_code = onto.VASP
first_input = some_code.mandatory_input[0]
appears_elsewhere = first_input.generic_parameter[0].has_parameters
can_come_from = first_input.get_sources(first_input.get_all_conditions())
which_is_produced_by = can_come_from[0].output_of
print('some_code', some_code)
print('first_input', first_input)
print('appears_elsewhere', appears_elsewhere)
print('can_come_from', can_come_from)
print('which_is_produced_by', which_is_produced_by)

some_code pyiron_atomistics.VASP
first_input pyiron_atomistics.VASP/input/structure
appears_elsewhere [pyiron_atomistics.CreateStructureBulk/output/structure, pyiron_atomistics.CreateSurface/output/structure, pyiron_atomistics.VASP/input/structure, pyiron_atomistics.LAMMPS/input/structure]
can_come_from [pyiron_atomistics.CreateStructureBulk/output/structure]
which_is_produced_by [pyiron_atomistics.CreateStructureBulk]


This is powerful, but can be a bit unwieldly. 

`pyiron_ontology` also comes with helper tool for building this sort of chain, or "workflow" automatically.

In [8]:
from pyiron_ontology import build_atomistics_tree as build_tree

First, let's see all the possible chains for getting input to a Lammps calculation:

In [9]:
build_tree(onto.LAMMPS).render()

pyiron_atomistics.LAMMPS
	pyiron_atomistics.LAMMPS/input/structure
		pyiron_atomistics.CreateStructureBulk/output/structure
			pyiron_atomistics.CreateStructureBulk
				pyiron_atomistics.CreateStructureBulk/input/element
		pyiron_atomistics.CreateSurface/output/structure
			pyiron_atomistics.CreateSurface
				pyiron_atomistics.CreateSurface/input/element


This tool also passes conditions transitively down. For instance, we see above that Lammps can take either bulk-like or non-bulk-like structure input. Instead of querying the ontology about what's needed to run a particular code, let's ask for a workflow to produce a particular material property: the bulk modulus. In this case, we know the workflow only makes sense if the structures going into it are bulk-like!

When we ask for this workflow, we again see Lammps coming up as part of our tree, but now we see that it is precluded from taking surface structures because the condition for a bulk-like structure got passed down through our workflow!

In [10]:
build_tree(onto.Bulk_modulus).render()

pyiron_atomistics.Bulk_modulus
	pyiron_atomistics.Murnaghan/output/equilibrium_bulk_modulus
		pyiron_atomistics.Murnaghan
			pyiron_atomistics.Murnaghan/ref_job
				pyiron_atomistics.VASP
					pyiron_atomistics.VASP/input/structure
				pyiron_atomistics.LAMMPS
					pyiron_atomistics.LAMMPS/input/structure


We also have tools for leveraging the ontology to search through existing pyiron data in your storage and database 

Here we'll need import `pyiron_atomistics.Project` so we can create some data to work with.

In [11]:
from pyiron_ontology import AtomisticsReasoner
from pyiron_atomistics import Project
import numpy as np

In [12]:
reasoner = AtomisticsReasoner(onto) 

Next, we'll produce some data and then use the a tool on the reasoner to search over it for data that matches a particular ontological property.

First, we'll need to produce some data to search over. In this case, let's calculate the bulk modulus for a couple of alloys with varying Nickle content. On a single-core laptop, this might take two or three minutes.

In [13]:
pr = Project('example')
pr.remove_jobs(recursive=True)

Are you sure you want to delete all jobs from 'example'? y/(n) y


In [14]:
for host in ['Al', 'Cu']:
    for frac in [0., 0.10, 0.25]:
        ref = pr.atomistics.job.Lammps(f"Lammps_host{host}_frac{frac:.2f}".replace(".", "d"))
        ref.structure = pr.atomistics.structure.bulk(host, cubic=True).repeat(3)
        random_ids = np.random.choice(
            np.arange(len(ref.structure), dtype=int), 
            int(frac * len(ref.structure))
        )
        ref.structure[random_ids] = "Ni"
        ref.potential = ref.list_potentials()[0]

        murn = pr.atomistics.job.Murnaghan(f"Murn_host{host}_frac{frac:.2f}".replace(".", "d"))
        murn.input['num_points']=7
        murn.ref_job = ref
        murn.run()

The job Murn_hostAl_frac0d00 was saved and received the ID: 316
The job Murn_hostAl_frac0d00_0_9 was saved and received the ID: 317
The job Murn_hostAl_frac0d00_0_9333333 was saved and received the ID: 318
The job Murn_hostAl_frac0d00_0_9666667 was saved and received the ID: 319
The job Murn_hostAl_frac0d00_1_0 was saved and received the ID: 320
The job Murn_hostAl_frac0d00_1_0333333 was saved and received the ID: 321
The job Murn_hostAl_frac0d00_1_0666667 was saved and received the ID: 322
The job Murn_hostAl_frac0d00_1_1 was saved and received the ID: 323
The job Murn_hostAl_frac0d10 was saved and received the ID: 324
The job Murn_hostAl_frac0d10_0_9 was saved and received the ID: 325
The job Murn_hostAl_frac0d10_0_9333333 was saved and received the ID: 326
The job Murn_hostAl_frac0d10_0_9666667 was saved and received the ID: 327
The job Murn_hostAl_frac0d10_1_0 was saved and received the ID: 328
The job Murn_hostAl_frac0d10_1_0333333 was saved and received the ID: 329
The job Murn_h

2023-01-25 12:46:20,393 - pyiron_log - WARNING - Minimum could not be found!


The job Murn_hostCu_frac0d00 was saved and received the ID: 340
The job Murn_hostCu_frac0d00_0_9 was saved and received the ID: 341
The job Murn_hostCu_frac0d00_0_9333333 was saved and received the ID: 342
The job Murn_hostCu_frac0d00_0_9666667 was saved and received the ID: 343
The job Murn_hostCu_frac0d00_1_0 was saved and received the ID: 344
The job Murn_hostCu_frac0d00_1_0333333 was saved and received the ID: 345
The job Murn_hostCu_frac0d00_1_0666667 was saved and received the ID: 346
The job Murn_hostCu_frac0d00_1_1 was saved and received the ID: 347
The job Murn_hostCu_frac0d10 was saved and received the ID: 348
The job Murn_hostCu_frac0d10_0_9 was saved and received the ID: 349
The job Murn_hostCu_frac0d10_0_9333333 was saved and received the ID: 350
The job Murn_hostCu_frac0d10_0_9666667 was saved and received the ID: 351
The job Murn_hostCu_frac0d10_1_0 was saved and received the ID: 352
The job Murn_hostCu_frac0d10_1_0333333 was saved and received the ID: 353
The job Murn_h

Now let's search the pyiron database for instances of some of our physically-meaningful properties:

In [15]:
reasoner.search_database_for_property(onto.Bulk_modulus, pr)

,Chemical Formula,Bulk_modulus [MPa],Engine
0,Al108,81146.527225,Lammps
1,Al99Ni9,85624.607595,Lammps
2,Al83Ni25,NaN,Lammps
3,Cu108,141955.804050,Lammps
4,Cu98Ni10,147202.959841,Lammps
5,Cu84Ni24,149854.281709,Lammps


We can also filter our search by chemistry:

In [16]:
reasoner.search_database_for_property(onto.B_prime, pr, select_alloy="Cu")

,Chemical Formula,B_prime [1],Engine
0,Cu108,4.394886,Lammps
1,Cu98Ni10,4.372170,Lammps
2,Cu84Ni24,3.697843,Lammps


# Cleanup

In [17]:
pr.remove_jobs_silently(recursive=True)
pr.remove(enable=True)

<ipython-input-17-18562755bd1e>:1: DeprecationWarning: pyiron_base.project.generic.remove_jobs_silently is deprecated: Use pr.remove_jobs(silently=True) rather than pr.remove_jobs_silently()..
  pr.remove_jobs_silently(recursive=True)


  0%|          | 0/48 [00:00<?, ?it/s]